In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../FNR_fake_processed.csv')
df

,Unnamed: 0,title,text,subject,date,all
0,0,Donald Trump Sends Out Embarrassing New Years ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Donald Trump Sends Out Embarrassing New Years ...
1,1,Drunk Bragging Trump Staffer Started Russian C...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Drunk Bragging Trump Staffer Started Russian C...
2,2,Sheriff David Clarke Becomes An Internet Joke ...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Sheriff David Clarke Becomes An Internet Joke ...
3,3,Trump Is So Obsessed He Even Has Obamas Name C...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Trump Is So Obsessed He Even Has Obamas Name C...
4,4,Pope Francis Just Called Out Donald Trump Duri...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Pope Francis Just Called Out Donald Trump Duri...
...,...,...,...,...,...,...
23476,23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",McPain: John McCain Furious That Iran Treated ...
23477,23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478,23478,Sunnistan: US and Allied Safe Zone Plan to Tak...,Patrick Henningsen 21st Century WireRemember w...,Middle-east,"January 15, 2016",Sunnistan: US and Allied Safe Zone Plan to Tak...
23479,23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",How to Blow $700 Million: Al Jazeera America F...


In [4]:
len(df['subject'].unique())

6

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
all_vectorizer = TfidfVectorizer()
X = all_vectorizer.fit_transform(df['all'])
len(all_vectorizer.get_feature_names_out())

80451

In [16]:
X.shape

(23481, 80451)

In [11]:
from sklearn.cluster import KMeans